## Import

In [1]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, box
import geopandas as gpd
import ee
import geemap
from rasterio import features
import json
import os
# ee.Authenticate()
ee.Initialize()

In [4]:
img_path1 = "/Users/pranavprakash/Library/CloudStorage/OneDrive-AiDashInc/Documents/data/change_detection_dataset/dominion_labeled_2022/Image/ID_6_20211109.tif"
# img_path2 = "/Users/pranavprakash/Library/CloudStorage/OneDrive-AiDashInc/Documents/data/change_detection_dataset/dominion_labeled_2022/Image/ID_3_20220401.tif"
raster1 = rasterio.open(img_path1)
# raster2 = rasterio.open(img_path2)
img_bounds = raster1.bounds
img_bound_geom = box(*img_bounds)
bounds_dict = {"id":[1], "geometry": img_bound_geom}
df_bounds = gpd.GeoDataFrame(bounds_dict, crs = str(raster1.crs))
df_bounds = df_bounds.to_crs('EPSG:4326')
df_bounds.to_file("temp/data_shp/id_3/id3.shp")

In [51]:
img_r = raster1.read(1)
mask = img_r != 0
shapes = features.shapes(mask.astype(np.uint8), transform=raster1.transform)
geom = []
values = []
for g, v in shapes:
    values.append(v)
    geom.append(g)
values = np.array(values)
geom = np.array(geom)
predicted_array = values == 1.00
geom = geom[predicted_array]
values = values[predicted_array]
geom_poly = list(map(lambda x: Polygon(x['coordinates'][0]), geom))
df_bounds = gpd.GeoDataFrame({"values": values}, crs= str(raster1.crs), geometry = geom_poly)
df_bounds = df_bounds.to_crs('EPSG:4326')

## Sentinel 2 images

In [ ]:
def maskS2clouds(image):
    qa = image.select('QA60')

    ## Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and (qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(4095)

In [68]:
coords = [[-80.03016465770727, 35.79131089623311],
   [-80.02917480544455, 35.87260726788011],
   [-80.14549748050524, 35.87349106731046],
   [-80.14636876781381, 35.79219207332858],
   [-80.03016465770727, 35.79131089623311]]

In [26]:
# image_before_start = '2016-05-01'
image_before_start = '2019-12-01'
image_before_end = '2020-12-01'
DW_bands = [
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
DW_partial_bands = ['water', 'trees', 'grass', 'built']
S2_bands = ['B4', 'B3', 'B2', 'B8', 'B11']
featureCollection = ee.FeatureCollection([ee.Feature(json.loads(df_bounds.to_json())['features'][0])])
geom = ee.Geometry(ee.FeatureCollection([ee.Feature(json.loads(df_bounds.to_json())['features'][0])]).geometry())
# geom = ee.Geometry.Polygon(json.loads(df_bounds.to_json())['features'][0]['geometry']['coordinates'][0])
image_col = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").\
            filterDate(image_before_start, image_before_end)\
            .filterBounds(geom)\
            .select(S2_bands)\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))\
                
# image_col = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").\
#             filterDate(image_before_start, image_before_end)\
#             .filterBounds(geom)\
#                 .select(DW_partial_bands)
# image = image_col.mosaic().clip(geom)   

In [27]:
print(image_col.size().getInfo())

46


In [28]:
geemap.ee_export_image_collection(image_col, out_dir="temp/s2_id6", scale = 10, region = geom)
# geemap.ee_export_image_collection_to_drive(image_col, folder="dw_world_id3", scale = 10, region = geom)

Total number of images: 46

Exporting 1/46: 20191205T160641_20191205T160803_T17SNV.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/pranavprakash/Documents/aidash/change_detection/temp/s2_id6/20191205T160641_20191205T160803_T17SNV.tif


Exporting 2/46: 20191205T160641_20191205T160803_T17SPV.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/pranavprakash/Documents/aidash/change_detection/temp/s2_id6/20191205T160641_20191205T160803_T17SPV.tif


Exporting 3/46: 20191215T160651_20191215T160836_T17SNV.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/pranavprakash/Documents/aidash/change_detection/temp/s2_id6/20191215T160651_20191215T160836_T17SNV.tif


Exporting 4/46: 20191215T160651_20191215T160836_T17SPV.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/pranavprakash/Documents/aidash/change_detection/temp/s2_id6/20191215T160651_20191215T160836_T17SPV.tif


Exporting 5/46: 20191220T160649_20191220T160804_T17SNV.tif
Generating UR

## Download images from shapefile

In [1]:
import geopandas as gpd
import pandas as pd

In [58]:
gdf_all = gpd.read_file("/Users/pranavprakash/Documents/aidash/data/ameren/pipeline_regions/regions.shp")
gdf = gdf_all.iloc[0:1]
gdf = gdf.to_crs("EPSG:4326")
bounds = gdf.bounds.values[0]
bounds_dict = {"id":[1], "geometry": box(*bounds)}
df = gpd.GeoDataFrame(bounds_dict, crs = "EPSG:4326")


In [60]:
year_start = 2021
month_start = 1
year_end = 2023
month_end = 3
image_before_start = '{}-{}-01'.format(str(year_start).zfill(2), str(month_start).zfill(2))
image_before_end = '{}-{}-01'.format(str(year_end).zfill(2), str(month_end).zfill(2))
DW_bands = [
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
DW_partial_bands = ['water', 'trees', 'grass', 'built']
S2_bands = ['B4', 'B3', 'B2', 'B8', 'B11']
featureCollection = ee.FeatureCollection([ee.Feature(json.loads(df.to_json())['features'][0])])
geom = ee.Geometry(ee.FeatureCollection([ee.Feature(json.loads(df.to_json())['features'][0])]).geometry())
# geom = ee.Geometry.Polygon(json.loads(df_bounds.to_json())['features'][0]['geometry']['coordinates'][0])
image_col = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").\
            filterDate(image_before_start, image_before_end)\
            .filterBounds(geom)\
            .select(DW_bands)

In [61]:
image_col.size()

In [62]:
image = image_col.first()
image.clip(geom).getInfo()['bands'][8]

{'id': 'snow_and_ice',
 'data_type': {'type': 'PixelType', 'precision': 'double'},
 'dimensions': [1426, 3054],
 'origin': [4134, 6661],
 'crs': 'EPSG:32615',
 'crs_transform': [10, 0, 753950, 0, 10, 4490420]}

In [63]:
from rasterio.warp import calculate_default_transform
from rasterio.windows import Window

In [64]:
image_info = image.clip(geom).getInfo()
src_height = image_info['bands'][0]['dimensions'][0]
src_width = image_info['bands'][0]['dimensions'][1]
crs = image_info['bands'][0]['crs']
crs_transform = image_info['bands'][0]['crs_transform']
dst_crs = 'EPSG:4326'
transform, width, height = calculate_default_transform(
        crs, dst_crs, src_width, src_height, *df.to_crs(crs).bounds.values[0])
transformer = rasterio.transform.AffineTransformer(transform)

In [81]:
rasterio.transform.Affine(*crs_transform)*(0,0)

(753950.0, 4490420.0)

In [66]:
df.to_crs(crs).bounds.values[0], crs_transform

(array([ 795295.00651412, 4557036.82869472,  823394.34859817,
        4587568.73855679]),
 [10, 0, 753950, 0, 10, 4490420])

In [67]:
(src_width, width), (src_height, height)

((3054, 2757), (1426, 2246))

In [68]:
from rasterio.features import bounds

In [69]:
bounds(box(*df.bounds.values[0]))

(-89.46901278935663, 41.1105179138407, -89.14867124376941, 41.3751863080569)

In [70]:
df.bounds

,minx,miny,maxx,maxy
0,-89.469013,41.110518,-89.148671,41.375186


In [71]:
transform*(0,0)

(-89.48322522777792, 41.3854588139214)

In [72]:
(transform*(0,0))[0], (transform*(0+width,0+height))[1], (transform*(0+width,0+height))[0], (transform*(0+width,0))[1]

(-89.48322522777792, 41.10022843776771, -89.1331004962304, 41.3854588139214)

In [73]:

windw_bounds = (transform*(0,0))[0], (transform*(0+1024,0+1024))[1], (transform*(0+1024,0+1024))[0], (transform*(0+1024,0))[1]
img_bound_geom = box(*windw_bounds)

In [74]:
img_bound_geom.area

0.0169111039369331

In [79]:
def get_geom_at_window(transform, col_off, row_off, width, height, dst_crs):
    windw_bounds = (transform*(col_off,row_off))[0], (transform*(col_off,row_off+height))[1], (transform*(col_off+width,row_off+height))[0], (transform*(col_off,row_off))[1]
    img_bound_geom = box(*windw_bounds)
    bounds_dict = {"id":[1], "geometry": img_bound_geom}
    df_bounds = gpd.GeoDataFrame(bounds_dict, crs = dst_crs)
    geom = ee.Geometry(ee.FeatureCollection([ee.Feature(json.loads(df_bounds.to_json())['features'][0])]).geometry())
    return geom
tile_width, tile_height = 256, 256
source_dir = "temp/ameren_dw_geom_cropped_1"
year = 2023
month = 1
end_dates_per_month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
while((year<year_end) or (month<month_end)):
    print(year, month)
    month_dir = source_dir+"/{}/{}".format(str(year).zfill(2), str(month).zfill(2))
    for x_idx in range(0, width, tile_width):
        for y_idx in range(0, height, tile_height):
            geom = get_geom_at_window(transform, x_idx, y_idx, tile_width, tile_height, dst_crs)
            out_dir = month_dir+"/tile_{}_{}".format(str(y_idx), str(x_idx))
            os.makedirs(out_dir, exist_ok=True)
            
            image_start = '{}-{}-01'.format(str(year).zfill(2), str(month).zfill(2))
            image_end = '{}-{}-{}'.format(str(year).zfill(2), str(month).zfill(2), str(end_dates_per_month[month]))
            image_col_monthly = image_col.filterDate(image_start, image_end)
            # geemap.ee_export_image_collection(image_col_monthly, out_dir=out_dir, scale = 10, region = geom)
            if(os.path.exists(out_dir+"/img.tif")): continue
            geemap.ee_export_image(image_col_monthly.median(), filename=out_dir+"/img.tif", scale = 10, region = geom)
    year = year+(month)//12
    month = (month)%12+1


2023 1
Generating URL ...
Please wait ...
Data downloaded to /Users/pranavprakash/Documents/aidash/change_detection/temp/ameren_dw_geom_cropped_1/2023/01/tile_0_0/img.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/pranavprakash/Documents/aidash/change_detection/temp/ameren_dw_geom_cropped_1/2023/01/tile_256_0/img.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/pranavprakash/Documents/aidash/change_detection/temp/ameren_dw_geom_cropped_1/2023/01/tile_512_0/img.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/pranavprakash/Documents/aidash/change_detection/temp/ameren_dw_geom_cropped_1/2023/01/tile_768_0/img.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/pranavprakash/Documents/aidash/change_detection/temp/ameren_dw_geom_cropped_1/2023/01/tile_1024_0/img.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/pranavprakash/Documents/aidash/change_detection/temp/ameren_dw_geom_cropped_1/2023/01/tile_1280_0/img.t

UnboundLocalError: local variable 'r' referenced before assignment

In [17]:
image_col.filterBounds(geom).first().clip(geom)

In [23]:
properties = image.propertyNames()
properties.getInfo()

['system:version',
 'system:id',
 'DATATAKE_IDENTIFIER',
 'AOT_RETRIEVAL_ACCURACY',
 'SPACECRAFT_NAME',
 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
 'CLOUD_SHADOW_PERCENTAGE',
 'MEAN_SOLAR_AZIMUTH_ANGLE',
 'system:footprint',
 'VEGETATION_PERCENTAGE',
 'SOLAR_IRRADIANCE_B12',
 'SOLAR_IRRADIANCE_B10',
 'SENSOR_QUALITY',
 'SOLAR_IRRADIANCE_B11',
 'GENERATION_TIME',
 'SOLAR_IRRADIANCE_B8A',
 'FORMAT_CORRECTNESS',
 'CLOUD_COVERAGE_ASSESSMENT',
 'THIN_CIRRUS_PERCENTAGE',
 'system:time_end',
 'WATER_VAPOUR_RETRIEVAL_ACCURACY',
 'system:time_start',
 'DATASTRIP_ID',
 'PROCESSING_BASELINE',
 'SENSING_ORBIT_NUMBER',
 'NODATA_PIXEL_PERCENTAGE',
 'SENSING_ORBIT_DIRECTION',
 'GENERAL_QUALITY',
 'GRANULE_ID',
 'REFLECTANCE_CONVERSION_CORRECTION',
 'MEDIUM_PROBA_CLOUDS_PERCENTAGE',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8',
 'DATATAKE_TYPE',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B6',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B7',
 'MEAN_INCIDENCE_AZIMUTH_

In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
id3_df = gpd.read_file("/Users/pranavprakash/Library/CloudStorage/OneDrive-AiDashInc/Documents/data/change_detection_dataset/dominion_labeled_2022/labels/ID_3_20220401")
id6_df = gpd.read_file("/Users/pranavprakash/Library/CloudStorage/OneDrive-AiDashInc/Documents/data/change_detection_dataset/dominion_labeled_2022/labels/ID_6_20211109")
id8_df = gpd.read_file("/Users/pranavprakash/Library/CloudStorage/OneDrive-AiDashInc/Documents/data/change_detection_dataset/dominion_labeled_2022/labels/ID_8_20220401")
id9_df = gpd.read_file("/Users/pranavprakash/Library/CloudStorage/OneDrive-AiDashInc/Documents/data/change_detection_dataset/dominion_labeled_2022/labels/ID_9_122020")

In [36]:
id3_df = id3_df.rename({'TimeLabel': 'Time label'}, axis=1)
id9_df = id9_df.rename({'Status': 'Time label'}, axis=1)
id3_df = id3_df.replace("Under contruction Sites", "Under construction Sites")
id6_df = id6_df.replace("Construction of Swimming Pool", "Construction of swimming pool")
id6_df = id6_df.replace("Under contruction Sites", "Under construction Sites")
id9_df = id9_df.replace("Under contruction Sites", "Under construction Sites")

In [37]:
df_concat = gpd.GeoDataFrame(pd.concat([id3_df, id6_df, id8_df, id9_df], ignore_index=True))

In [38]:
print(len(id3_df)+len(id6_df)+len(id8_df)+len(id9_df), len(df_concat))
print(df_concat.Label.unique())

7519 7519
['Clearing of timberland' 'Buildings' 'Construction of new roads'
 'Removal of structures' 'Construction of swimming pool'
 'Clearing for foundations' 'Others' 'Under construction Sites'
 'Water bodies']


In [39]:
df_concat.to_file("/Users/pranavprakash/Library/CloudStorage/OneDrive-AiDashInc/Documents/data/change_detection_dataset/dominion_labeled_2022/labels/full_labels/full_label.shp")

In [35]:
id3_df.Label.unique(), id6_df.Label.unique(), id8_df.Label.unique(), id9_df.Label.unique()

(array(['Clearing of timberland', 'Buildings', 'Construction of new roads',
        'Removal of structures', 'Construction of swimming pool',
        'Clearing for foundations', 'Others', 'Under contruction Sites',
        'Water bodies'], dtype=object),
 array(['Construction of Swimming Pool', 'Buildings',
        'Construction of new roads', 'Water bodies',
        'Removal of structures', 'Clearing for foundations',
        'Under contruction Sites', 'Clearing of timberland'], dtype=object),
 array(['Construction of new roads', 'Clearing of timberland',
        'Removal of structures', 'Buildings',
        'Construction of swimming pool', 'Clearing for foundations',
        'Under construction Sites', 'Water bodies'], dtype=object),
 array(['Removal of structures', 'Construction of new roads', 'Buildings',
        'Water bodies', 'Clearing for foundations',
        'Construction of swimming pool', 'Clearing of timberland',
        'Others', 'Under contruction Sites'], dtype=object))